In [132]:
import glob
import pandas as pd
from collections import Counter

In [188]:
#Read artists master data that will be joined to concatenated list of all social data
artists = pd.read_csv('master_artists_list.csv')

In [86]:
artists.columns

Index(['artist_name', 'age', 'years_active', 'genre_pop', 'genre_rock',
       'genre_hip_hop', 'genre_world_music', 'genre_r_and_b',
       'genre_spiritual', 'genre_electronic', 'genre_jazz', 'genre_country',
       'genre_blues', 'genre_folk', 'facebook', 'instagram', 'twitter',
       'youtube'],
      dtype='object')

In [87]:
#Read all social data files in
files = glob.glob('SocialData/*.csv')
files

['SocialData\\facebook_7yrs.csv',
 'SocialData\\instagram_7yrs_cleaned.csv',
 'SocialData\\twitter_features.csv',
 'SocialData\\youtube_to_combine.csv']

In [88]:
df_list = [pd.read_csv(file) for file in files]

C:\Users\oldye\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3214: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [108]:
sum = 0
for d in df_list:
    print("num columns", len(d.columns))
    sum+=d.shape[0]
    print(d.shape)
    print(d.columns)

print("number of rows in 4 dataframes: ", sum)

num columns 26
(297603, 26)
Index(['Unnamed: 0', 'artist_name', 'created', 'description',
       'crowdtangle_score', 'count_of_followers', 'type_link', 'type_photo',
       'type_text', 'type_video', 'description_length', 'created_12am-6am',
       'created_12pm-6pm', 'created_6am-12pm', 'created_6pm-12am',
       'time_since_last_post', 'hashtag_count', 'within_week_release',
       'within_month_release', 'social_engagement_score', 'posts_last_7days',
       'posts_last_30days', 'Facebook', 'Instagram', 'Twitter', 'Youtube'],
      dtype='object')
num columns 26
(385381, 26)
Index(['Unnamed: 0', 'artist_name', 'count_of_followers', 'created',
       'description', 'crowdtangle_score', 'description_length',
       'hashtag_count', 'social_engagement_score', 'type_photo', 'type_video',
       'created_12am-6am', 'created_12pm-6pm', 'created_6am-12pm',
       'created_6pm-12am', 'time_since_last_post', 'type_text', 'type_link',
       'posts_last_7days', 'posts_last_30days', 'within_we

In [90]:
#Temporary fixes for youtube
df_list[3].drop(columns='time_of_day', inplace=True)
df_list[3].reset_index(inplace=True)
df_list[3].rename(columns = {'index': 'Unnamed: 0'}, inplace=True)

In [91]:
#Renaming columns that were not aligned
#df_list[0].rename(columns = {'fb_handle_artist_name': 'artist_name'}, inplace=True) #Facebook
df_list[2].rename(columns = {'Twitter_Handle': 'artist_name'}, inplace=True) #Twitter

In [92]:
#Adding Dummy columns to indicate which dataset it is
df_list[0]['Facebook'] = 1
df_list[0]['Instagram'] = 0
df_list[0]['Twitter'] = 0
df_list[0]['Youtube'] = 0

df_list[1]['Facebook'] = 0
df_list[1]['Instagram'] = 1
df_list[1]['Twitter'] = 0
df_list[1]['Youtube'] = 0

df_list[2]['Facebook'] = 0
df_list[2]['Instagram'] = 0
df_list[2]['Twitter'] = 1
df_list[2]['Youtube'] = 0

df_list[3]['Facebook'] = 0
df_list[3]['Instagram'] = 0
df_list[3]['Twitter'] = 0
df_list[3]['Youtube'] = 1

In [189]:
fb = df_list[0].merge(artists, left_on='artist_name', right_on='facebook')
instagram = df_list[1].merge(artists, left_on='artist_name', right_on='instagram')
twitter = df_list[2].merge(artists, left_on='artist_name', right_on='twitter')
yt = df_list[3].merge(artists, left_on='artist_name', right_on='youtube')
socials = [fb, instagram, twitter, yt]

In [190]:
for s in socials:
    print(s.shape)

(297603, 44)
(385381, 44)
(290677, 44)
(20149, 44)


###### Investigation Start

In [174]:
fb_check = df_list[0].merge(artists, how='outer', left_on='artist_name', right_on='facebook')


In [175]:
fb_check.shape

(297603, 44)

In [176]:
#Delta between full outer join and inner join
fb_check.shape[0] - fb.shape[0]

0

In [178]:
fb_diff = pd.concat([fb, fb_check]).drop_duplicates(keep=False)

In [179]:
fb_diff.head()

,Unnamed: 0,artist_name_x,created,description,crowdtangle_score,count_of_followers,type_link,type_photo,type_text,type_video,...,genre_spiritual,genre_electronic,genre_jazz,genre_country,genre_blues,genre_folk,facebook,instagram,twitter,youtube


In [180]:
#Differences match
fb_diff.shape

(0, 44)

In [168]:
fb_diff2 = fb_diff[['artist_name_x', 'facebook', 'instagram', 'twitter', 'youtube']]

In [169]:
#NOTE: THIS is what was missing originally.  Will be blank if re-run
fb_diff2.groupby('artist_name_x').min()

,facebook,instagram,twitter,youtube
artist_name_x,,,,
AliceCooper,NaN,NaN,NaN,NaN
PearlJam,NaN,NaN,NaN,NaN
RickyMartinOfficialPage,NaN,NaN,NaN,NaN
Scorpions,NaN,NaN,NaN,NaN
SiaMusic,NaN,NaN,NaN,NaN
WillieNelson,NaN,NaN,NaN,NaN
fifthharmony,NaN,NaN,NaN,NaN
mirandalambert,NaN,NaN,NaN,NaN
swedishhousemafia,NaN,NaN,NaN,NaN


##### Instagram Check

In [191]:
in_check = df_list[1].merge(artists, how='outer', left_on='artist_name', right_on='instagram')
print(in_check.shape)
#Delta between full outer join and inner join
print(in_check.shape[0] - instagram.shape[0])

in_diff = pd.concat([instagram, in_check]).drop_duplicates(keep=False)

(385381, 44)
0


In [186]:
in_diff.head()

,Unnamed: 0,artist_name_x,count_of_followers,created,description,crowdtangle_score,description_length,hashtag_count,social_engagement_score,type_photo,...,genre_spiritual,genre_electronic,genre_jazz,genre_country,genre_blues,genre_folk,facebook,instagram,twitter,youtube
54968,177844.0,cncomusic,NaN,2015-12-13 23:17:04,¡Bienvenidos a nuestra cuenta oficial! Double ...,-3.2,107.0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54969,177851.0,cncomusic,NaN,2015-12-14 00:01:00,¡Gracias @ricky_martin @laurapausini y @alejan...,-4.24,126.0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54970,177862.0,cncomusic,NaN,2015-12-14 01:28:18,We made it! 🎤🎼🎉🎊 #CNCO #DevuelvemeMiCorazon,-4.57,43.0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54971,177886.0,cncomusic,NaN,2015-12-14 10:05:26,¡Estamos en @despiertaamericatv no te lo pierd...,-4.91,76.0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54972,177888.0,cncomusic,NaN,2015-12-14 10:23:17,Gracias @despiertaamericatv por su apoyo🎤 #CNC...,-5.42,68.0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [192]:
#Differences match
in_diff.shape

(0, 44)

In [187]:
in_diff2 = in_diff[['artist_name_x', 'facebook', 'instagram', 'twitter', 'youtube']]
in_diff2.groupby('artist_name_x').min()

,facebook,instagram,twitter,youtube
artist_name_x,,,,
cncomusic,NaN,NaN,NaN,NaN
ptxofficial,NaN,NaN,NaN,NaN
swedishhousemafia,NaN,NaN,NaN,NaN
threedaysgraceofficial,NaN,NaN,NaN,NaN


###### investigation stop

In [193]:
df = pd.concat(socials)
df.drop('Unnamed: 0',axis=1,inplace=True)

C:\Users\oldye\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [96]:
df.sample(10)

,Facebook,Instagram,Twitter,Youtube,age,artist_name_x,artist_name_y,count_of_followers,created,created_12am-6am,...,time_since_last_post,twitter,type_link,type_photo,type_text,type_video,within_month_release,within_week_release,years_active,youtube
201963,0,1,0,0,25,maluma,Maluma,33287939.0,2018-06-16 01:54:05,1,...,0 days 00:24:54.000000000,maluma,0,1,0,0,0,0,9,Maluma
360374,0,1,0,0,40,usher,Usher,3152639.0,2015-10-09 04:50:59,1,...,0 days 00:09:09.000000000,Usher,0,1,0,0,0,0,27,Usher
175714,0,0,1,0,51,celinedion,Celine Dion,408424.0,2015-11-16 21:05:27,0,...,0 days 00:23:12.000000000,celinedion,1,0,0,0,0,0,39,CelineDion
20482,1,0,0,0,34,FrenchMontanaOfficial,French Montana,4192710.0,11/27/16 20:00,0,...,0 days 00:44:58.000000000,FrencHMonTanA,1,0,0,0,0,0,17,frenchmontana
232010,0,0,1,0,25,maluma,Maluma,3153043.0,2016-09-04 16:55:37,0,...,0 days 00:02:06.000000000,maluma,1,0,0,0,0,0,9,Maluma
249705,1,0,0,0,28,thechainsmokers,The Chainsmokers,NaN,11/8/12 18:58,0,...,0 days 16:49:16.000000000,TheChainsmokers,0,1,0,0,0,0,7,The Chainsmokers
68198,1,0,0,0,38,NickyJamPR,Nicky Jam,NaN,3/2/15 2:00,1,...,0 days 00:06:13.000000000,NickyJamPR,0,1,0,0,0,0,25,NickyJamTV
174924,0,1,0,0,44,juicyj,Juicy J,NaN,2014-11-21 08:00:21,0,...,0 days 15:53:15.000000000,therealjuicyj,0,0,0,1,0,0,28,JuicyJ
121587,0,1,0,0,65,francodevita,Franco De Vita,1042368.0,2017-02-04 13:23:33,0,...,0 days 22:39:40.000000000,FrancoDeVita,0,0,0,1,0,0,37,Franco De Vita
105063,1,0,0,0,47,Thalia,Thalia,13915407.0,10/17/14 15:46,0,...,0 days 10:37:02.000000000,thalia,0,1,0,0,0,0,38,Thalia


In [194]:
print(df.shape)
print("number of rows lost: ", sum - df.shape[0], ", ", (sum - df.shape[0])/sum)


(993810, 43)
number of rows lost:  0 ,  0.0


In [196]:
df.to_csv('Combined_Social_Data.csv')